In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from answer_judge.vqaEval import VQAEval

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/VQA/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answers,question_id,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[Is the window open?\nAnswer the question usin...,"[{'answer': ['yes', 'wood', 'daytime', 'beige'...","[tensor(526696001), tensor(526696002), tensor(...",{'embedding': [[[ 0.03491211 -0.00491333 -0.02...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Are the boats all the same color?\nAnswer the...,"[{'answer': ['no', 'shoes', 'carpet', 'beige',...","[tensor(404003), tensor(262549000), tensor(262...",{'embedding': [[[-0.00189972 0.01818848 -0.00...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[How many floors does the bus have?\nAnswer th...,"[{'answer': ['2', 'yellow', 'yes', 'yes', 'yes...","[tensor(136900003), tensor(136900004), tensor(...",{'embedding': [[[ 1.6937256e-03 -7.1048737e-05...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[How many animals are there?\nAnswer the quest...,"[{'answer': ['6', 'yes', 'yes', '2', 'yes', 'n...","[tensor(133741001), tensor(133741002), tensor(...",{'embedding': [[[-0.00122833 0.00131989 -0.01...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[What shape is the rug?\nAnswer the question u...,"[{'answer': ['round', 'red', 'yes', 'red', 'gr...","[tensor(357921002), tensor(357921003), tensor(...",{'embedding': [[[ 0.01013184 0.00717163 -0.00...,None


In [5]:
df.shape

(2500, 6)

In [6]:
i=0

In [7]:
df.iloc[i]['img_path']

['/root/autodl-fs/coco_images/train/COCO_train2014_000000526696.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000526696.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000526696.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000526696.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000526696.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000526696.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000526696.jpg',
 '/root/autodl-fs/coco_images/train/COCO_train2014_000000526697.jpg']

In [8]:
df.iloc[i]['most_likely']['response']

['Yes', 'Metal', 'Daytime', 'White', 'Yes', 'Yes', '1', '5']

In [9]:
df.iloc[i]['question']

['Is the window open?\nAnswer the question using a single word or phrase.\n',
 'What is the barrier next to the toilet made of?\nAnswer the question using a single word or phrase.\n',
 'Is it daytime or night time outside?\nAnswer the question using a single word or phrase.\n',
 'What color are the wall tiles?\nAnswer the question using a single word or phrase.\n',
 'Does the toilet touch the ground?\nAnswer the question using a single word or phrase.\n',
 'Could a person fix their makeup in this room?\nAnswer the question using a single word or phrase.\n',
 'How many towels are shown?\nAnswer the question using a single word or phrase.\n',
 'How many boards are there?\nAnswer the question using a single word or phrase.\n']

In [10]:
df.iloc[i]['answers']

[{'answer': ['yes', 'wood', 'daytime', 'beige', 'no', 'yes', '2', '6'],
  'answer_confidence': ['yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'maybe',
   'yes',
   'yes'],
  'answer_id': tensor([1, 1, 1, 1, 1, 1, 1, 1])},
 {'answer': ['yes', 'wood', 'daytime', 'white', 'yes', 'yes', '2', '7'],
  'answer_confidence': ['maybe',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'maybe',
   'maybe'],
  'answer_id': tensor([2, 2, 2, 2, 2, 2, 2, 2])},
 {'answer': ['yes', 'wood', 'daytime', 'white', 'yes', 'yes', '5', '7'],
  'answer_confidence': ['yes',
   'yes',
   'maybe',
   'yes',
   'yes',
   'maybe',
   'maybe',
   'yes'],
  'answer_id': tensor([3, 3, 3, 3, 3, 3, 3, 3])},
 {'answer': ['yes', 'wood', 'daytime', 'gray', 'no', 'no', '2', '7'],
  'answer_confidence': ['yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'yes',
   'maybe',
   'maybe'],
  'answer_id': tensor([4, 4, 4, 4, 4, 4, 4, 4])},
 {'answer': ['yes', 'wood', 'daytime', 'brown', 'yes', 'no', '0', '7'],
  'answer_conf

In [11]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [12]:
embeddings=np.concatenate(embeddings,axis=0)

In [13]:
embeddings.shape

(20000, 33, 4096)

In [14]:
def transpose_lines(x):
    f=pd.DataFrame(x)
    transposed=f['answer'].apply(pd.Series).T.values.tolist()
    return transposed

df['answers']=df['answers'].apply(transpose_lines)

In [15]:
gt_label=df['answers'].to_list()
gt_label=np.concatenate(gt_label,axis=0).tolist()

In [16]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [17]:
rsps[0]

'Yes'

In [18]:
judge=VQAEval(gt_label,rsps)
judge.evaluate()
print(judge.evalQA)
print(judge.accuracy['overall'])

computing accuracy


  0%|          | 0/20000 [00:00<?, ?it/s]

100%|██████████| 20000/20000 [00:06<00:00, 3017.53it/s]

Done computing accuracy
{0: 100, 1: 0.0, 2: 100, 3: 100, 4: 100, 5: 100, 6: 66.67, 7: 0.0, 8: 100, 9: 33.33, 10: 100, 11: 100, 12: 100, 13: 100, 14: 100, 15: 100, 16: 100, 17: 100, 18: 100, 19: 100, 20: 100, 21: 100, 22: 33.33, 23: 100, 24: 0.0, 25: 100, 26: 100, 27: 33.33, 28: 100, 29: 100, 30: 100, 31: 100, 32: 100, 33: 100, 34: 100, 35: 100, 36: 33.33, 37: 100, 38: 66.67, 39: 100, 40: 100, 41: 100, 42: 100, 43: 100, 44: 100, 45: 100, 46: 66.67, 47: 0.0, 48: 100, 49: 100, 50: 100, 51: 100, 52: 100, 53: 66.67, 54: 100, 55: 100, 56: 0.0, 57: 66.67, 58: 100, 59: 100, 60: 100, 61: 0.0, 62: 100, 63: 100, 64: 100, 65: 66.67, 66: 100, 67: 100, 68: 100, 69: 66.67, 70: 100, 71: 100, 72: 100, 73: 0.0, 74: 100, 75: 100, 76: 100, 77: 100, 78: 100, 79: 100, 80: 100, 81: 0.0, 82: 66.67, 83: 100, 84: 100, 85: 0.0, 86: 100, 87: 100, 88: 100, 89: 100, 90: 100, 91: 100, 92: 100, 93: 33.33, 94: 100, 95: 100, 96: 100, 97: 100, 98: 100, 99: 100, 100: 66.67, 101: 100, 102: 100, 103: 100, 104: 100, 105: 10

In [19]:
gt_scores=[judge.evalQA[i] for i in range(len(judge.evalQA))]

In [20]:
is_correct=[]
threshold=30
for i,x in enumerate(gt_scores):
    if x >=threshold:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [21]:
is_correct.shape

(20000,)

In [22]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 20000 responses. 16833 answer correctly and 3167 answer wrongly


In [45]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 2000
# exclude validation samples.
train_idxs = train_val_idxs[:- validset_len]  # trainset index
val_idxs = train_val_idxs[- validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:14000
validset length:2000
testset length:4000
trainset postive: 11792 negative: 2208
valset postive: 1667 negative: 333
testset postive: 3374 negative: 626


In [46]:
all_embeddings = embeddings
all_embeddings.shape

(20000, 33, 4096)

In [47]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [48]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [49]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCALinear(X[:,i_layer,:],n_components)
    scores=discriminator.get_score(X[:,i_layer,:])
    
    return roc_auc_score(y,scores)
    #split=discriminator.get_best_split(scores,y)
    #y_pred=scores>split
    #return accuracy_score(y,y_pred)
    #return balanced_accuracy_score(y,y_pred)
    #
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [50]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,3),
    'layer':range(10,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

found best n_components:1,layer:10 Score: 0.5930525606590393
found best n_components:1,layer:11 Score: 0.5990153320687215
found best n_components:1,layer:12 Score: 0.6000277421993079
found best n_components:1,layer:13 Score: 0.6075775835823827
found best n_components:1,layer:14 Score: 0.6145671766547591
found best n_components:1,layer:15 Score: 0.6444657014543038


  0%|          | 0/23 [00:00<?, ?it/s]

found best n_components:2,layer:15 Score: 0.6497745495945855


In [51]:
best_paras

{'n_components': 2, 'layer': 15, 'best_score': np.float64(0.6497745495945855)}

In [52]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [53]:
discriminator=PCALinear(embedding_val[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [54]:
discriminator=PCALinear(embedding_train[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.8332857142857143 AUROC:0.6507647118287023


In [55]:
discriminator=PCALinear(embedding_test[:,best_layer,:],best_paras['n_components'])
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.80975 AUROC:0.6546919120278923


Train Linear Prob to Get Better Results

In [56]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [57]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [58]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores)}')

test set acc: 0.83025 f1:0.9068459322266429 auroc:0.6546919120278923
